# MIMIC 4 data - dataset construction labevents

Code taken from GRU-ODE-Bayes preprocessing; simplified and adapted for MIMIC 4 1.0

In [ ]:
import gzip
from pathlib import Path
from zipfile import ZipFile

import pandas as pd
import pyarrow
import pyarrow.csv
import pyarrow.parquet

# Load `labevents` and `labitems` tables

## Table Schema

In [ ]:
rawdata_file = Path.cwd() / "mimic-iv-1.0.zip"
dataset_path = Path.cwd() / "processed"
rawdata_path = Path.cwd() / "raw"

files = {
    "labevents": "mimic-iv-1.0/hosp/labevents.csv.gz",
    "labitems": "mimic-iv-1.0/hosp/d_labitems.csv.gz",
}

CATEGORY = pyarrow.dictionary("int32", "string")

column_types = {
    "labevents": {
        "labevent_id": "int32",
        "subject_id": "int32",
        "hadm_id": "int32",
        "specimen_id": "int32",
        "itemid": "int32",
        "charttime": "timestamp[s]",
        "storetime": "timestamp[s]",
        "value": CATEGORY,
        "valuenum": "float32",
        "valueuom": CATEGORY,
        "ref_range_lower": "float32",
        "ref_range_upper": "float32",
        "flag": CATEGORY,
        "priority": CATEGORY,
        "comments": CATEGORY,
    },
    "labitems": {
        "itemid": "int32",
        "label": CATEGORY,
        "fluid": CATEGORY,
        "category": CATEGORY,
        "loinc_code": CATEGORY,
    },
}

## Load `labevents` table

In [ ]:
# shape: (120M, 15) ⇝ 1.8B values.
key = "labevents"
with (
    ZipFile(rawdata_file) as archive,
    archive.open(files[key]) as compressed_file,
    gzip.open(compressed_file) as file,
):
    labevents = pyarrow.csv.read_csv(
        file,
        convert_options=pyarrow.csv.ConvertOptions(
            column_types=column_types[key],
            strings_can_be_null=True,
            null_values=[""],
        ),
    )

labevents.schema

## Store and reload as pandas

In [ ]:
pyarrow.parquet.write_table(labevents, rawdata_path / f"{key}.parquet")

labevents = labevents.to_pandas(self_destruct=True)
labevents

## Load `labitems` table

In [ ]:
# shape: (120M, 15) ⇝ 1.8B values.
key = "labitems"
with (
    ZipFile(rawdata_file) as archive,
    archive.open(files[key]) as compressed_file,
    gzip.open(compressed_file) as file,
):
    labitems = pyarrow.csv.read_csv(
        file,
        convert_options=pyarrow.csv.ConvertOptions(
            column_types=column_types[key],
            strings_can_be_null=True,
            null_values=[""],
        ),
    )

labitems.schema

## Store and reload as pandas

In [ ]:
pyarrow.parquet.write_table(labitems, rawdata_path / f"{key}.parquet")

labitems = labitems.to_pandas(self_destruct=True)
labitems

# Filter Dataset

## Only keep labevents associated with the selected patients

In [ ]:
admissions = pd.read_parquet(dataset_path / "admissions_processed.parquet")
labevents = labevents[labevents.hadm_id.isin(admissions.hadm_id)]
print(f"Number of events remaining: {labevents.shape}")
print(f"Number of patients remaining: {labevents.subject_id.nunique()}")

## Merge on `ItemID`, only select tests that were given to many people

In [ ]:
labevents = pd.merge(labevents, labitems, on="itemid")

In [ ]:
selected_variables = [
    "Albumin",
    "Alanine Aminotransferase (ALT)",
    "Alkaline Phosphatase",
    "Anion Gap",
    "Asparate Aminotransferase (AST)",
    "Base Excess",
    "Basophils",
    "Bicarbonate",
    "Bilirubin, Total",
    "Calcium, Total",
    "Calculated Total CO2",
    "Chloride",
    "Creatinine",
    "Eosinophils",
    "Glucose",
    "Hematocrit",
    "Hemoglobin",
    "Lactate",
    "Lymphocytes",
    "MCH",
    "MCV",
    "Magnesium",
    "Monocytes",
    "Neutrophils",
    "PT",
    "PTT",
    "Phosphate",
    "Platelet Count",
    "Potassium",
    "RDW",
    "Red Blood Cells",
    "Sodium",
    "Specific Gravity",
    "Urea Nitrogen",
    "White Blood Cells",
    "pCO2",
    "pH",
    "pO2",
]

labevents = labevents[labevents["label"].isin(selected_variables)]
print(f"Number of patients remaining: {labevents.subject_id.nunique()}")

In [ ]:
# test_freq = labevents.groupby("label")["subject_id"].nunique()
# common_tests = test_freq.sort_values(ascending=False).index[:150]
# labevents = labevents[labevents["label"].isin(common_tests)]
# print(f"Number of patients remaining: {labevents.subject_id.nunique()}")

# Serialize Pre-processed DataFrame

In [ ]:
# Clean categories
def clean_categories(df):
    for col in df:
        if df[col].dtype == "category":
            df[col] = df[col].cat.remove_unused_categories()
    return df


labevents = clean_categories(labevents)
labevents.to_parquet(dataset_path / "labevents_processed.parquet")